# Let's familiarize ourselves with the ``Network`` object

__...by interactively building the elements of the network one by one__

In this tutorial we explore the following :

1. Network parameters
2. __Initialize all the elements of the network__ using  the ``Network.build_*`` methods
    1. Models (neuron models, recorder models, synapse models
    2. Layers
    3. Projection models
    4. Individual projections
    5. Population and projection recorders
3. __Create the network__ in NEST
4. __Access the network elements__ (GIDs, etc)
4. Export and reuse the parameter tree allowing us to __replicate the network__

In [1]:
%load_ext autoreload
%autoreload 2

import nest
import yaml
from pathlib import Path
from pprint import pprint

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from denest import *
import denest

In [3]:
PARAMS_DIR = Path('./data_tuto/params')
OUTPUT_DIR = Path('./data_tuto/output')

## 1- Initialize an empty network

When initialized without argument or with an empty tree as an argument, all the expected subtrees are initialized as empty.

When building the elements interactively, the network's parameters are updated

In [4]:
net = Network()

2020-06-25 16:00:35,875 [denest.utils.validation] INFO: 'None' tree: adding empty child neuron_models
2020-06-25 16:00:35,877 [denest.utils.validation] INFO: 'None' tree: adding empty child synapse_models
2020-06-25 16:00:35,878 [denest.utils.validation] INFO: 'None' tree: adding empty child layers
2020-06-25 16:00:35,879 [denest.utils.validation] INFO: 'None' tree: adding empty child projection_models
2020-06-25 16:00:35,880 [denest.utils.validation] INFO: 'None' tree: adding empty child topology
2020-06-25 16:00:35,882 [denest.utils.validation] INFO: 'None' tree: adding empty child recorder_models
2020-06-25 16:00:35,883 [denest.utils.validation] INFO: 'None' tree: adding empty child recorders
2020-06-25 16:00:35,885 [denest.network] INFO: Build N=0 ``Model`` objects
2020-06-25 16:00:35,886 [denest.network] INFO: Build N=0 ``SynapseModel`` objects
2020-06-25 16:00:35,889 [denest.network] INFO: Build N=0 ``Model`` objects
2020-06-25 16:00:35,890 [denest.network] INFO: Build N=0 ``Laye

In [5]:
# Network parameter tree is empty:
net.tree

ParamsTree(name='None', parent=None)
  params: {}
  nest_params: {}
  neuron_models:
    params: {}
    nest_params: {}
  synapse_models:
    params: {}
    nest_params: {}
  layers:
    params: {}
    nest_params: {}
  projection_models:
    params: {}
    nest_params: {}
  topology:
    params: {}
    nest_params: {}
  recorder_models:
    params: {}
    nest_params: {}
  recorders:
    params: {}
    nest_params: {}
  

## 2- Build the network components (models, layers, projections, recorders)


### Define new models

We can define neuron, recorder, stimulator and synapse models with arbitrary parameters from parameter trees.

Each leaf corresponds to a new (named) model. Its ``nest_params`` and ``params`` are hierarchically inherited.

The ``nest_model`` used is specified in the leaf's ``params``

#### ``'neuron_models'`` tree:

Initalize ``Network.neuron_models`` with the ``Network.build_neuron_models`` method

In [6]:
neuron_models_tree = ParamsTree.read(PARAMS_DIR/'models.yml').children['neuron_models']
pprint(neuron_models_tree)

ParamsTree(name='neuron_models', parent='None')
  params: {}
  nest_params: {}
  my_neuron:
    params:
      nest_model: ht_neuron
    nest_params:
      g_KL: 1.0
      g_NaL: 1.0
    l1_exc:
      params: {}
      nest_params:
        V_m: -44.0
    l1_inh:
      params: {}
      nest_params:
        V_m: -55.0
  


In [7]:
net.build_neuron_models(neuron_models_tree)

2020-06-25 16:00:36,113 [denest.network] INFO: Build N=2 ``Model`` objects


In [8]:
# The neuron models are saved as an attribute for the Network object
print("\n``Network.neuron_models`` :")
pprint(net.neuron_models)


``Network.neuron_models`` :
{'l1_exc': Model(l1_exc, {'nest_model': 'ht_neuron'}, {'V_m': -44.0, 'g_KL': 1.0, 'g_NaL': 1.0}),
 'l1_inh': Model(l1_inh, {'nest_model': 'ht_neuron'}, {'V_m': -55.0, 'g_KL': 1.0, 'g_NaL': 1.0})}


In [9]:
pprint(net.neuron_models['l1_exc'])
pprint(net.neuron_models['l1_exc'].params)
pprint(net.neuron_models['l1_exc'].nest_params)

Model(l1_exc, {'nest_model': 'ht_neuron'}, {'V_m': -44.0, 'g_KL': 1.0, 'g_NaL': 1.0})
{'nest_model': 'ht_neuron'}
{'V_m': -44.0, 'g_KL': 1.0, 'g_NaL': 1.0}


#### ``'recorder_models'`` tree:

Initalize ``Network.recorder_models`` with the ``Network.build_recorder_models`` method

-> Same thing as for neuron models

In [10]:
# ``Network.build_*`` methods accept as argument ``ParamsTree`` objects, but also tree-like dictionaries 
recorder_models_tree = ParamsTree.read(PARAMS_DIR/'models.yml').children['recorder_models']
recorder_models_tree

ParamsTree(name='recorder_models', parent='None')
  params: {}
  nest_params:
    record_to:
    - memory
    - file
  weight_recorder:
    params:
      nest_model: weight_recorder
    nest_params: {}
  my_multimeter:
    params:
      nest_model: multimeter
    nest_params:
      record_from:
      - V_m
  my_spike_detector:
    params:
      nest_model: spike_detector
    nest_params: {}
  

In [11]:
# ``Network.build_*`` methods accept as argument ``ParamsTree`` objects, but also tree-like dictionaries 
recorder_models_tree = recorder_models_tree.asdict()
recorder_models_tree

{'params': {},
 'nest_params': {'record_to': ['memory', 'file']},
 'weight_recorder': {'params': {'nest_model': 'weight_recorder'},
  'nest_params': {}},
 'my_multimeter': {'params': {'nest_model': 'multimeter'},
  'nest_params': {'record_from': ['V_m']}},
 'my_spike_detector': {'params': {'nest_model': 'spike_detector'},
  'nest_params': {}}}

In [12]:
net.build_recorder_models(recorder_models_tree)

2020-06-25 16:00:36,448 [denest.network] INFO: Build N=3 ``Model`` objects


In [13]:
print("\n``Network.recorder_models`` :")
pprint(net.recorder_models)


``Network.recorder_models`` :
{'my_multimeter': Model(my_multimeter, {'nest_model': 'multimeter'}, {'record_from': ['V_m'], 'record_to': ['memory', 'file']}),
 'my_spike_detector': Model(my_spike_detector, {'nest_model': 'spike_detector'}, {'record_to': ['memory', 'file']}),
 'weight_recorder': Model(weight_recorder, {'nest_model': 'weight_recorder'}, {'record_to': ['memory', 'file']})}


#### ``'synapse_model'`` tree:

Initalize ``Network.synapse_models`` with the ``Network.build_synapse_model`` method

-> Same thing as for neuron models, with as a bonus a convenient way of specifying the receptor type of the synapse

-> If specifying the ``receptor_type`` and ``target_model`` in the ``SynapseModel`` params, the corresponding port is determined automatically

In [14]:
synapse_models_tree = ParamsTree.read(PARAMS_DIR/'models.yml').children['synapse_models']
synapse_models_tree

ParamsTree(name='synapse_models', parent='None')
  params: {}
  nest_params: {}
  my_AMPA_synapse:
    params:
      nest_model: ht_synapse
      receptor_type: AMPA
      target_neuron: ht_neuron
    nest_params: {}
  my_GABAA_synapse:
    params:
      nest_model: ht_synapse
      receptor_type: GABA_A
      target_neuron: ht_neuron
    nest_params: {}
  

In [15]:
net.build_synapse_models(synapse_models_tree)

2020-06-25 16:00:36,626 [denest.network] INFO: Build N=2 ``SynapseModel`` objects


In [16]:
print("\n``Network.synapse_models`` :")
pprint(net.synapse_models)


``Network.synapse_models`` :
{'my_AMPA_synapse': SynapseModel(my_AMPA_synapse, {'nest_model': 'ht_synapse'}, {'receptor_type': 1}),
 'my_GABAA_synapse': SynapseModel(my_GABAA_synapse, {'nest_model': 'ht_synapse'}, {'receptor_type': 3})}


Note that the ``receptor_type`` nest_parameter was inferred

### Define layers

As for models, we can create ``nest.Topology`` layers from the leaves of a tree.

The elements can be nest models with their default parameters, or the ones we just created with custom params

For layers of stimulator devices, we can use the ``InputLayer`` object, which can automatically create paired parrot neurons for each stimulator units, by adding ``type: 'InputLayer'``
to the params


#### ``'layers'`` tree:


In [17]:
layer_tree = ParamsTree.read(PARAMS_DIR/'layers.yml')
layer_tree

ParamsTree(name='None', parent=None)
  params: {}
  nest_params: {}
  layers:
    params:
      type: null
    nest_params:
      rows: 5
      columns: 5
      extent:
      - 5.0
      - 5.0
      edge_wrap: true
    input_layer:
      params:
        type: InputLayer
        add_parrots: true
        populations:
          spike_generator: 1
      nest_params: {}
    l1:
      params:
        populations:
          l1_exc: 4
          l1_inh: 2
      nest_params: {}
  

In [18]:
net.build_layers(layer_tree)

2020-06-25 16:00:36,822 [denest.network] INFO: Build N=2 ``Layer`` or ``InputLayer`` objects.


In [19]:
pprint(net.layers)

{'input_layer': InputLayer(input_layer, {'add_parrots': True,
 'populations': {'parrot_neuron': 1, 'spike_generator': 1},
 'type': 'InputLayer'}, {'columns': 5,
 'edge_wrap': True,
 'elements': ('spike_generator', 1, 'parrot_neuron', 1),
 'extent': [5.0, 5.0],
 'rows': 5}),
 'l1': Layer(l1, {'populations': {'l1_exc': 4, 'l1_inh': 2}, 'type': None}, {'columns': 5,
 'edge_wrap': True,
 'elements': ('l1_exc', 4, 'l1_inh', 2),
 'extent': [5.0, 5.0],
 'rows': 5})}


In [20]:
print("'l1' layer")
pprint(net.layers['l1'].params)
pprint(net.layers['l1'].nest_params)

'l1' layer
{'populations': {'l1_exc': 4, 'l1_inh': 2}, 'type': None}
{'columns': 5,
 'edge_wrap': True,
 'elements': ('l1_exc', 4, 'l1_inh', 2),
 'extent': [5.0, 5.0],
 'rows': 5}


### Define projections

We create projections using a two step process:

1. Create ``ProjectionModel`` objects from a tree. Each named leaf will define a template from which individual projections can inherit their parameters
2. Create ``Projection`` objects from a list, specifying for each item the source layer x population, target layer x population and the projection model to inherit parameters from

#### 1- Define templates from the `projection_models` tree



In [21]:
proj_model_tree = ParamsTree.read(PARAMS_DIR/'projections.yml').children['projection_models']
proj_model_tree

ParamsTree(name='projection_models', parent='None')
  params: {}
  nest_params:
    connection_type: divergent
    mask:
      circular:
        radius: 2.0
    kernel: 1.0
  proj_1_AMPA:
    params: {}
    nest_params:
      synapse_model: my_AMPA_synapse
      weights: 1.0
  proj_2_GABAA:
    params: {}
    nest_params:
      synapse_model: my_GABAA_synapse
      weights: 2.0
  

In [22]:
net.build_projection_models(proj_model_tree)

2020-06-25 16:00:37,064 [denest.utils.validation] INFO: Object `proj_1_AMPA`: params: using default value for optional parameters:
{'type': 'topological'}
2020-06-25 16:00:37,075 [denest.utils.validation] INFO: Object `proj_2_GABAA`: params: using default value for optional parameters:
{'type': 'topological'}
2020-06-25 16:00:37,076 [denest.network] INFO: Build N=2 ``ProjectionModel`` objects


In [23]:
net.projection_models

{'proj_1_AMPA': ProjectionModel(proj_1_AMPA, 
  {'type': 'topological'}{'connection_type': 'divergent',
   'mask': {'circular': {'radius': 2.0}},
   'kernel': 1.0,
   'synapse_model': 'my_AMPA_synapse',
   'weights': 1.0}),
 'proj_2_GABAA': ProjectionModel(proj_2_GABAA, 
  {'type': 'topological'}{'connection_type': 'divergent',
   'mask': {'circular': {'radius': 2.0}},
   'kernel': 1.0,
   'synapse_model': 'my_GABAA_synapse',
   'weights': 2.0})}

### 2- Define individual projections from the `topology` tree

The list of projections is defined in the `projections` params of the `topology` tree

Check out the doc of `Network.build_projections` for expected formatting

In [24]:
conns_tree = ParamsTree.read(PARAMS_DIR/'projections.yml').children['topology']
conns_tree

ParamsTree(name='topology', parent='None')
  params:
    projections:
    - source_layers:
      - input_layer
      source_population: parrot_neuron
      target_layers:
      - l1
      target_population: l1_exc
      projection_model: proj_1_AMPA
    - source_layers:
      - l1
      source_population: l1_exc
      target_layers:
      - l1
      target_population: l1_inh
      projection_model: proj_1_AMPA
    - source_layers:
      - l1
      source_population: l1_inh
      target_layers:
      - l1
      target_population: l1_exc
      projection_model: proj_2_GABAA
  nest_params: {}
  

In [25]:
net.projection_models

{'proj_1_AMPA': ProjectionModel(proj_1_AMPA, 
  {'type': 'topological'}{'connection_type': 'divergent',
   'mask': {'circular': {'radius': 2.0}},
   'kernel': 1.0,
   'synapse_model': 'my_AMPA_synapse',
   'weights': 1.0}),
 'proj_2_GABAA': ProjectionModel(proj_2_GABAA, 
  {'type': 'topological'}{'connection_type': 'divergent',
   'mask': {'circular': {'radius': 2.0}},
   'kernel': 1.0,
   'synapse_model': 'my_GABAA_synapse',
   'weights': 2.0})}

In [26]:
net.build_projections(conns_tree)

2020-06-25 16:00:37,319 [denest.network] INFO: Build N=3 ``TopoProjection`` objects


In [27]:
net.projections

[TopoProjection(proj_1_AMPA-input_layer-parrot_neuron-l1-l1_exc, 
  {'type': 'topological'}{'connection_type': 'divergent',
   'mask': {'circular': {'radius': 2.0}},
   'kernel': 1.0,
   'synapse_model': 'my_AMPA_synapse',
   'weights': 1.0,
   'sources': {'model': 'parrot_neuron'},
   'targets': {'model': 'l1_exc'}}),
 TopoProjection(proj_1_AMPA-l1-l1_exc-l1-l1_inh, 
  {'type': 'topological'}{'connection_type': 'divergent',
   'mask': {'circular': {'radius': 2.0}},
   'kernel': 1.0,
   'synapse_model': 'my_AMPA_synapse',
   'weights': 1.0,
   'sources': {'model': 'l1_exc'},
   'targets': {'model': 'l1_inh'}}),
 TopoProjection(proj_2_GABAA-l1-l1_inh-l1-l1_exc, 
  {'type': 'topological'}{'connection_type': 'divergent',
   'mask': {'circular': {'radius': 2.0}},
   'kernel': 1.0,
   'synapse_model': 'my_GABAA_synapse',
   'weights': 2.0,
   'sources': {'model': 'l1_inh'},
   'targets': {'model': 'l1_exc'}})]

### Define recorders from the `recorders` tree

Similarly to the `topology` tree, recorders are defined from lists.


We separate recorders connected to synapses (eg weight recorder) and those connected to units (eg spike detectors),
which are defined in the `projection_recorders` and `population_recorders` params (resp.) of the `recorders` tree.


Check out the doc of the ``Network.build_recorders``, ``Network.build_population_recorders`` and
``Network.build_projection_recorders`` methods for expected formatting

The parameters of the recorders can be changed by using custom recorder models (in the `recorder_models` tree, see above) 

In [28]:
recorders_tree = ParamsTree.read(PARAMS_DIR/'recorders.yml').children['recorders']
recorders_tree

ParamsTree(name='recorders', parent='None')
  params:
    population_recorders:
    - layers:
      - l1
      populations:
      - l1_exc
      model: my_multimeter
    - layers:
      - input_layer
      populations: null
      model: my_spike_detector
    projection_recorders:
    - source_layers:
      - l1
      source_population: l1_exc
      target_layers:
      - l1
      target_population: l1_inh
      projection_model: proj_1_AMPA
      model: weight_recorder
  nest_params: {}
  

In [29]:
net.build_recorders(recorders_tree)

2020-06-25 16:00:37,553 [denest.network] INFO: Build N=2 population recorders.
2020-06-25 16:00:37,555 [denest.network] INFO: Build N=1 projection recorders.


In [30]:
net.population_recorders

[PopulationRecorder(my_multimeter_l1_l1_exc,  {}{}),
 PopulationRecorder(my_spike_detector_input_layer_parrot_neuron,  {}{})]

In [31]:
net.projection_recorders

[ProjectionRecorder(weight_recorder_proj_1_AMPA-l1-l1_exc-l1-l1_inh,  {}{})]

## 3- Create the network

In [32]:
nest.ResetKernel()
nest.SetKernelStatus({'overwrite_files': True})

In [33]:
net.create()

2020-06-25 16:00:37,837 [denest.network] INFO: Creating neuron models...
100%|██████████| 2/2 [00:00<00:00, 1867.46it/s]
2020-06-25 16:00:37,851 [denest.network] INFO: Creating synapse models...
100%|██████████| 2/2 [00:00<00:00, 382.33it/s]
2020-06-25 16:00:37,861 [denest.network] INFO: Creating recorder models...
100%|██████████| 3/3 [00:00<00:00, 96.31it/s]
2020-06-25 16:00:37,906 [denest.network] INFO: Creating layers...
  0%|          | 0/2 [00:00<?, ?it/s]/Users/tom/nest/nest-simulator-2.20.0/lib/python3.7/site-packages/nest/lib/hl_api_helper.py:127: UserWarning:
GetNodes is deprecated and will be removed in NEST 3.0. Use             GIDCollection instead.
100%|██████████| 2/2 [00:00<00:00,  7.96it/s]
2020-06-25 16:00:38,165 [denest.network] INFO: Creating population recorders...
100%|██████████| 2/2 [00:00<00:00, 89.09it/s]
2020-06-25 16:00:38,195 [denest.network] INFO: Creating projection recorders...
100%|██████████| 1/1 [00:00<00:00, 216.12it/s]
2020-06-25 16:00:38,206 [denes

## 4- Examine the network

denest provides convenient ways of accessing the objects in NEST

### Check the defaults of the created models

In [34]:
print("`l1_exc` neuron models `nest_params`: ", net.neuron_models['l1_exc'].nest_params)

`l1_exc` neuron models `nest_params`:  {'g_KL': 1.0, 'g_NaL': 1.0, 'V_m': -44.0}


In [35]:
net

Network(params: {}
nest_params: {}
neuron_models:
  params: {}
  nest_params: {}
  my_neuron:
    params:
      nest_model: ht_neuron
    nest_params:
      g_KL: 1.0
      g_NaL: 1.0
    l1_exc:
      params: {}
      nest_params:
        V_m: -44.0
    l1_inh:
      params: {}
      nest_params:
        V_m: -55.0
synapse_models:
  params: {}
  nest_params: {}
  my_AMPA_synapse:
    params:
      nest_model: ht_synapse
      receptor_type: AMPA
      target_neuron: ht_neuron
    nest_params: {}
  my_GABAA_synapse:
    params:
      nest_model: ht_synapse
      receptor_type: GABA_A
      target_neuron: ht_neuron
    nest_params: {}
layers:
  params: {}
  nest_params: {}
  layers:
    params:
      type: null
    nest_params:
      rows: 5
      columns: 5
      extent:
      - 5.0
      - 5.0
      edge_wrap: true
    input_layer:
      params:
        type: InputLayer
        add_parrots: true
        populations:
          spike_generator: 1
      nest_params: {}
    l1:
      para

In [36]:
print('Corresponding params of the `l1_exc` model in nest:', nest.GetDefaults('l1_exc', list(net.neuron_models['l1_exc'].nest_params.keys())))

Corresponding params of the `l1_exc` model in nest: (1.0, 1.0, -44.0)


### Access the layers' units

In [37]:
print('Layer `l1` shape: ', net.layers['l1'].layer_shape)
print('Population shapes: ', net.layers['l1'].population_shape)

Layer `l1` shape:  (5, 5)
Population shapes:  {'l1_exc': (5, 5, 4), 'l1_inh': (5, 5, 2)}


In [38]:
net.layers['l1'].gids(location=(0, 0), population='l1_exc')

[53, 78, 103, 128]

### Access the projections created in NEST

In [39]:
conn = net.projections[0]
conn

TopoProjection(proj_1_AMPA-input_layer-parrot_neuron-l1-l1_exc, 
{'type': 'topological'}{'connection_type': 'divergent',
 'mask': {'circular': {'radius': 2.0}},
 'kernel': 1.0,
 'synapse_model': 'my_AMPA_synapse',
 'weights': 1.0,
 'sources': {'model': 'parrot_neuron'},
 'targets': {'model': 'l1_exc'}})

In [40]:
nest_conns = nest.GetConnections(
    source=conn.source.gids(conn.source_population),
    target=conn.target.gids(conn.target_population),
    synapse_model=conn.nest_params['synapse_model']
)
nest_conns[0:5]

(array('l', [27, 53, 0, 68, 0]),
 array('l', [27, 88, 0, 68, 1]),
 array('l', [27, 83, 0, 68, 2]),
 array('l', [27, 152, 0, 68, 3]),
 array('l', [27, 84, 0, 68, 4]))

### Access the recorders

In [41]:
rec = net.population_recorders[0]
print(rec, rec.gid, rec.model, rec.layer, rec.population_name)

my_multimeter_l1_l1_exc (203,) my_multimeter l1 l1_exc


In [42]:
connrec = net.projection_recorders[0]
print(connrec, connrec.gid, connrec.model)

weight_recorder_proj_1_AMPA-l1-l1_exc-l1-l1_inh (205,) weight_recorder


## 5- Save and replicate the network

When building each of the network's elements using the `Network.build_*` methods, the `Network.tree` ParamsTree was updated

In [43]:
net.tree

ParamsTree(name='None', parent=None)
  params: {}
  nest_params: {}
  neuron_models:
    params: {}
    nest_params: {}
    my_neuron:
      params:
        nest_model: ht_neuron
      nest_params:
        g_KL: 1.0
        g_NaL: 1.0
      l1_exc:
        params: {}
        nest_params:
          V_m: -44.0
  
  ... [116 lines] ...

      - layers:
        - input_layer
        populations: null
        model: my_spike_detector
      projection_recorders:
      - source_layers:
        - l1
        source_population: l1_exc
        target_layers:
        - l1
        target_population: l1_inh
        projection_model: proj_1_AMPA
        model: weight_recorder
    nest_params: {}
  

#### We can save the parameter tree defining the whole network...

In [44]:
net.tree.write(PARAMS_DIR/'network_tree.yml')

PosixPath('data_tuto/params/network_tree.yml')

#### And use it to recreate another identical network

In [45]:
net2 = Network(ParamsTree.read(PARAMS_DIR/'network_tree.yml'))

2020-06-25 16:00:39,216 [denest.network] INFO: Build N=2 ``Model`` objects
2020-06-25 16:00:39,218 [denest.network] INFO: Build N=2 ``SynapseModel`` objects
2020-06-25 16:00:39,219 [denest.network] INFO: Build N=3 ``Model`` objects
2020-06-25 16:00:39,220 [denest.network] INFO: Build N=2 ``Layer`` or ``InputLayer`` objects.
2020-06-25 16:00:39,222 [denest.utils.validation] INFO: Object `proj_1_AMPA`: params: using default value for optional parameters:
{'type': 'topological'}
2020-06-25 16:00:39,226 [denest.utils.validation] INFO: Object `proj_2_GABAA`: params: using default value for optional parameters:
{'type': 'topological'}
2020-06-25 16:00:39,227 [denest.network] INFO: Build N=2 ``ProjectionModel`` objects
2020-06-25 16:00:39,234 [denest.network] INFO: Build N=3 ``TopoProjection`` objects
2020-06-25 16:00:39,238 [denest.network] INFO: Build N=2 population recorders.
2020-06-25 16:00:39,241 [denest.network] INFO: Build N=1 projection recorders.
2020-06-25 16:00:39,258 [denest.netw

In [46]:
net2

Network(params: {}
nest_params: {}
neuron_models:
  params: {}
  nest_params: {}
  my_neuron:
    params:
      nest_model: ht_neuron
    nest_params:
      g_KL: 1.0
      g_NaL: 1.0
    l1_exc:
      params: {}
      nest_params:
        V_m: -44.0
    l1_inh:
      params: {}
      nest_params:
        V_m: -55.0
synapse_models:
  params: {}
  nest_params: {}
  my_AMPA_synapse:
    params:
      nest_model: ht_synapse
      receptor_type: AMPA
      target_neuron: ht_neuron
    nest_params: {}
  my_GABAA_synapse:
    params:
      nest_model: ht_synapse
      receptor_type: GABA_A
      target_neuron: ht_neuron
    nest_params: {}
layers:
  params: {}
  nest_params: {}
  layers:
    params:
      type: null
    nest_params:
      rows: 5
      columns: 5
      extent:
      - 5.0
      - 5.0
      edge_wrap: true
    input_layer:
      params:
        type: InputLayer
        add_parrots: true
        populations:
          spike_generator: 1
      nest_params: {}
    l1:
      para